### Callback API
- 모델이 학습 중에 충돌이 발생하거나 네트워크가 끊기면, 모든 훈련 시간이 낭비될 수 있고,  
  과적합을 방지하기 위해 훈련을 중간에 중지해야 할 수도 있다.
- 모델이 학습을 시작하면 학습이 완료될 때까지 아무런 제어를 하지 못하게 되고,  
  신경망 훈련을 완료하는 데에는 몇 시간 또는 며칠이 걸릴 수 있기 때문에 모델을 모니터링하고 제어할 수 있는 기능이 필요하다.
- 훈련 시(fit()) Callback API를 등록시키면 반복 내에서 특정 이벤트 발생마다 등록된 callback이 호출되어 수행된다.

**ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weight_only=False, mode='auto')**
- 특정 조건에 따라서 모델 또는 가중치를 파일로 저장한다.
- filepath: "weight.{epoch: 03d}-{val_loss:.4f}-{acc:.4f}.weights.hdf5" 와 같이 모델의 체크포인트를 저장한다.
- monitor: 모니터링할 성능 지표를 작성한다.
- save_best_only: 가장 좋은 성능을 나타내는 모델을 저장할 지에 대한 여부
- save_weights_only: weights만 저장할 지에 대한 여부
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 좋은 경우를 선택한다.  
  *monitor의 성능 지표가 감소해야 좋은 경우 min, 증가해야 좋은 경우 max, monitor의 이름으로부터 자동으로 유추하고 싶다면 auto를 사용한다.

**ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto, min_lr=0')** (LR: Learning Rate)
- 특정 반복동안 성능이 개선되지 않을 때, 학습률을 동적으로 감소시킨다.
- monitor: 모니터링할 성능 지표를 작성한다.
- factor: 학습률을 감소시킬 비율, 새로운 학습률 = 기존 학습률 * factor
- patience: 학습률을 줄이기 전에 monitor할 반복 횟수
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 좋은 경우를 선택한다.  
  *monitor의 성능 지표가 감소해야 좋은 경우 min, 증가해야 좋은 경우 max, monitor의 이름으로부터 자동으로 유추하고 싶다면 auto를 사용한다.

**EarlyStopping(monitor='val_loss'm patient=0, verbose=0, mode='auto')**
- 특정 반복동안 성능이 개선되지 않을 때, 학습을 조기에 중단한다.
- monitor: 모니터링할 성능 지표를 작성한다.
- patience: Early Stopping을 적용하기 전에 monitor할 반복 횟수.
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 좋은 경우를 선택한다.  
  *monitor의 성능 지표가 감소해야 좋은 경우 min, 증가해야 좋은 경우 max, monitor의 이름으로부터 자동으로 유추하고 싶다면 auto를 사용한다.

In [1]:
from tensorflow.keras.layers import Layer, Input, Flatten, Dense
from tensorflow.keras.models import Model

INPUT_SIZE = 28

def create_model():
    input_tensor = Input(shape=(INPUT_SIZE, INPUT_SIZE))
    x = Flatten()(input_tensor)
    x = Dense(64, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(10, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output)
    return model

In [2]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

# 전처리

# 255 색상 minmax scale
def get_preprocessed_data(images, targets):
    images = np.array(images / 255.0, dtype = np.float32)
    targets = np.array(targets, dtype = np.float32)

    return images, targets

# 원핫 인코딩
def get_preprocessed_ohe(images, targets):
    images, targets = get_preprocessed_data(images, targets)
    oh_targets = to_categorical(targets)

    return images, oh_targets

def get_train_valid_test(train_images, train_targets, test_images, test_targets, validation_size=0.2, random_state=124):
    train_images, train_oh_targets= get_preprocessed_ohe(train_images, train_targets)
    test_images, test_oh_targets = get_preprocessed_ohe(test_images, test_targets)

    train_images, validation_images, train_oh_targets, validation_oh_targets =\
    train_test_split(train_images, train_oh_targets, stratify=train_oh_targets, test_size=validation_size, random_state=random_state)

    return (train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets)
    

In [3]:
from tensorflow.keras.datasets import fashion_mnist

(train_images, train_targets), (test_images, test_targets) = fashion_mnist.load_data()

(train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets) = \
get_train_valid_test(train_images, train_targets, test_images, test_targets)

print(train_images.shape, train_oh_targets.shape)
print(validation_images.shape, validation_oh_targets.shape)
print(test_images.shape, test_oh_targets.shape)

(48000, 28, 28) (48000, 10)
(12000, 28, 28) (12000, 10)
(10000, 28, 28) (10000, 10)


In [4]:
# !ls callback_files
!du -sh callback_files

  0B	callback_files


# ModelCheckPoint

In [11]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

mcp_cb = ModelCheckpoint(
    filepath="./callback_files/weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5",
    monitor='val_loss',
    # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True설정
    save_best_only=False,
    save_weights_only=True,
    mode='min'
)

# mcp_cb = ModelCheckpoint(
#     filepath="./callback_files/model.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.model.keras",
#     monitor='val_loss',
#     # 모든 epoch의 파일을 저장
#     save_best_only=False,
#     save_weights_only=False,
#     mode='min'
# )

history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[mcp_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 828us/step - acc: 0.7377 - loss: 0.7598 - val_acc: 0.8460 - val_loss: 0.4254
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 794us/step - acc: 0.8529 - loss: 0.4089 - val_acc: 0.8525 - val_loss: 0.3911
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 795us/step - acc: 0.8696 - loss: 0.3582 - val_acc: 0.8694 - val_loss: 0.3574
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 761us/step - acc: 0.8758 - loss: 0.3340 - val_acc: 0.8691 - val_loss: 0.3599
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 786us/step - acc: 0.8852 - loss: 0.3142 - val_acc: 0.8728 - val_loss: 0.3404
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 780us/step - acc: 0.8917 - loss: 0.2937 - val_acc: 0.8735 - val_loss: 0.3451
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 783us/step - acc: 0.8972 - loss: 0.2797 - val_acc: 0.8687 - val_loss: 0.3463
Epoch 8/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 751us/step - acc: 0.8993 - loss: 0.2713 - val_acc: 0.8745 - val_loss: 0.3391
Epoch 9/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 

In [12]:
!ls callback_files
# !du -sh callback_files

model.001-0.4776-0.8046.model.keras  weights.006-0.3404-0.8927.weights.h5
model.002-0.3782-0.8589.model.keras  weights.006-0.3451-0.8915.weights.h5
model.003-0.3573-0.8709.model.keras  weights.007-0.3217-0.8956.weights.h5
model.004-0.3578-0.8792.model.keras  weights.007-0.3463-0.8958.weights.h5
model.005-0.3657-0.8867.model.keras  weights.008-0.3342-0.8986.weights.h5
model.006-0.3250-0.8924.model.keras  weights.008-0.3391-0.9000.weights.h5
model.007-0.3305-0.8943.model.keras  weights.009-0.3164-0.9019.weights.h5
model.008-0.3318-0.8989.model.keras  weights.009-0.3182-0.9028.weights.h5
model.009-0.3243-0.9010.model.keras  weights.010-0.3277-0.9045.weights.h5
model.010-0.3160-0.9055.model.keras  weights.010-0.3378-0.9051.weights.h5
model.011-0.3270-0.9081.model.keras  weights.011-0.3129-0.9098.weights.h5
model.012-0.3285-0.9103.model.keras  weights.011-0.3144-0.9094.weights.h5
model.013-0.3281-0.9139.model.keras  weights.012-0.3184-0.9132.weights.h5
model.014-0.3277-0.9160.model.keras  w

In [13]:
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 322us/step - acc: 0.8838 - loss: 0.3760


[0.377580463886261, 0.8830000162124634]

In [15]:
model = create_model()
model.load_weights('./callback_files/weights.020-0.3434-0.9303.weights.h5')

model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 296us/step - acc: 0.8838 - loss: 0.3760


[0.377580463886261, 0.8830000162124634]

In [16]:
from tensorflow.keras.models import load_model

model = load_model('./callback_files/model.020-0.3346-0.9290.model.keras')
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 298us/step - acc: 0.8840 - loss: 0.3692


[0.3576537072658539, 0.8859000205993652]

# ReduceLROnPlateau

In [17]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])


# ReduceLROnPlateau 콜백 정의
rlr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    mode='min'
)



history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[rlr_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 906us/step - acc: 0.7380 - loss: 0.7534 - val_acc: 0.8522 - val_loss: 0.4091 - learning_rate: 0.0010
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 760us/step - acc: 0.8538 - loss: 0.4103 - val_acc: 0.8609 - val_loss: 0.3802 - learning_rate: 0.0010
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 870us/step - acc: 0.8673 - loss: 0.3626 - val_acc: 0.8652 - val_loss: 0.3570 - learning_rate: 0.0010
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 815us/step - acc: 0.8786 - loss: 0.3340 - val_acc: 0.8608 - val_loss: 0.3827 - learning_rate: 0.0010
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 788us/step - acc: 0.8845 - loss: 0.3131 - val_acc: 0.8767 - val_loss: 0.3335 - learning_rate: 0.0010
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 757us/step - acc: 0.8931 - loss: 0.2928 - val_acc: 0.8763 - val_loss: 0.3256 - learning_rate: 0.0010
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 963us/step - acc: 0.8949 - loss: 0.2907 - val_acc: 0.8767 - val_loss: 0.3276 - learning_rate:

# EarlyStopping

In [18]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])


# EarlyStopping 콜백 정의
ely_cb = EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min'
)

history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[ely_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 857us/step - acc: 0.7320 - loss: 0.7792 - val_acc: 0.8451 - val_loss: 0.4253
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 748us/step - acc: 0.8564 - loss: 0.3987 - val_acc: 0.8656 - val_loss: 0.3652
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 849us/step - acc: 0.8721 - loss: 0.3481 - val_acc: 0.8685 - val_loss: 0.3560
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 702us/step - acc: 0.8796 - loss: 0.3288 - val_acc: 0.8646 - val_loss: 0.3589
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 697us/step - acc: 0.8851 - loss: 0.3112 - val_acc: 0.8702 - val_loss: 0.3530
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 704us/step - acc: 0.8937 - loss: 0.2899 - val_acc: 0.8815 - val_loss: 0.3278
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 707us/step - acc: 0.8961 - loss: 0.2838 - val_acc: 0.8781 - val_loss: 0.3380
Epoch 8/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 713us/step - acc: 0.8990 - loss: 0.2720 - val_acc: 0.8832 - val_loss: 0.3269
Epoch 9/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 

# 3개 전부

In [19]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

mcp_cb = ModelCheckpoint(
    filepath="./callback_files/weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5",
    monitor='val_loss',
    # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True설정
    save_best_only=False,
    save_weights_only=True,
    mode='min'
)

rlr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    mode='min'
)

ely_cb = EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min'
)

history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[mcp_cb, rlr_cb, ely_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 847us/step - acc: 0.7428 - loss: 0.7482 - val_acc: 0.8481 - val_loss: 0.4186 - learning_rate: 0.0010
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 874us/step - acc: 0.8584 - loss: 0.3937 - val_acc: 0.8608 - val_loss: 0.3727 - learning_rate: 0.0010
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 730us/step - acc: 0.8742 - loss: 0.3476 - val_acc: 0.8702 - val_loss: 0.3514 - learning_rate: 0.0010
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 820us/step - acc: 0.8833 - loss: 0.3222 - val_acc: 0.8768 - val_loss: 0.3487 - learning_rate: 0.0010
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 875us/step - acc: 0.8862 - loss: 0.3036 - val_acc: 0.8760 - val_loss: 0.3408 - learning_rate: 0.0010
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 779us/step - acc: 0.8933 - loss: 0.2912 - val_acc: 0.8789 - val_loss: 0.3235 - learning_rate: 0.0010
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 815us/step - acc: 0.8978 - loss: 0.2795 - val_acc: 0.8789 - val_loss: 0.3271 - learning_rate: